In [1]:
import pdb
import os
import numpy as np
import pandas as pd
import pylab
import pylab as plt
from utils import find_nearest
from utils import find_nearest_index
from GameDayFunctions.fangraphs_projection import Projection
from GameDayFunctions.draft import Draft

%matplotlib inline

In [2]:
year = 2019
model = 'Steamer'
model = 'ZiPS'
path_data = '/data/baseball/Fangraphs/projections/'

In [3]:
ZiPS = Projection()

GameDayFunctions/fangraphs_projection.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.hitters_stats['Rank'][ind] = kk
GameDayFunctions/fangraphs_projection.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.pitchers_stats['Position'][ind] = fn
GameDayFunctions/fangraphs_projection.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.pitchers_stats['Rank'][ind] = k
GameDayFunctions/fangraphs_projection.py:93: SettingWithCopyWarning: 
A value is trying to be set on 

In [54]:
Steamer = Projection(model='Steamer')

In [5]:
ZiPS.precompute_statlines()

In [56]:
Steamer.precompute_statlines()

In [57]:
Steamer.statline['batters'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,playerid,Name,Drafted
0,96,87,38,4,40,111,3,64,0.292,0.937,579,3B,1,9777,Nolan Arenado,False
1,98,94,37,4,31,99,22,73,0.292,0.911,568,3B,2,13510,Jose Ramirez,False
2,99,93,38,3,29,93,11,74,0.286,0.892,568,3B,5,17678,Alex Bregman,False
3,94,86,34,3,31,93,7,78,0.274,0.884,563,3B,12,15429,Kris Bryant,False
4,89,100,40,2,24,87,4,67,0.293,0.875,566,3B,13,5235,Justin Turner,False


In [6]:
ZiPS.statline['batters'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,playerid,Name,Drafted
0,76,79,32,4,25,84,5,38,0.276,0.838,507,C,2,11739,J.T. Realmuto,False
1,55,53,19,1,23,69,1,68,0.238,0.811,403,C,4,11368,Yasmani Grandal,False
2,68,53,21,0,28,85,3,44,0.246,0.822,415,C,1,11442,Gary Sanchez,False
3,53,89,24,1,8,53,4,48,0.288,0.770,423,C,5,9166,Buster Posey,False
4,56,75,26,3,14,64,5,51,0.255,0.754,463,C,6,11609,Willson Contreras,False


In [49]:
class Draft:
    
    def __init__(self, projection, number_teams = 12,
                 batters  = {'C','1B','2B', '3B','SS','OF','UTIL'},
                 pitchers = {'SP','RP','P'},
                 number_positions = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}, 
                 batter_statline  = {'R':0,'1B':0,'2B':0, '3B':0,'HR':0,'RBI':0,'SB':0,'BB':0,'AVG':0,'OPS':0}, 
                 pitcher_statline = {'W':0, 'L':0,'CG':0,'SHO':0,'SV':0,'BB':0,'SO':0,'ERA':0,'WHIP':0,'BSV':0} ):
        
        self.projection = projection
        self.batters  = batters
        self.pitchers = pitchers
        self.open_positions = number_positions
        self.batter_statline = batter_statline
        self.pitcher_statline = pitcher_statline        
        self.number_teams = number_teams

        bat_rows = []
        bat_cols = ['Name']
        for ibat in self.batters:
            for i in range(self.open_positions[ibat]):
                bat_rows.append(ibat)
        for ibat in self.batter_statline:
            bat_cols.append(ibat)
        print(bat_rows)        
        print(bat_cols)
        
        bat_df = pd.DataFrame(np.zeros([len(bat_rows),len(bat_cols)]), columns = bat_cols)
        bat_df['Position'] = bat_rows
        
        pit_rows = ['Name']
        pit_cols = []
        for ipit in self.pitchers:
            for i in range(self.open_positions[ipit]):
                pit_rows.append(ipit)
        for ipit in self.pitcher_statline:
            pit_cols.append(ipit)
            
        print(pit_rows)        
        print(pit_cols)
        
        pit_df = pd.DataFrame(np.zeros([len(pit_rows),len(pit_cols)]), columns = pit_cols)
        pit_df['Position'] = pit_rows   
        
        self.bat_df = bat_df
        self.pit_df = pit_df
        
        #return bat_df, pit_df
    
    def draft_team(self, draft_position = 'Best'):
        #print(self.number_teams)
        self.draftees  = pd.DataFrame()
        for psn in self.batters:
            #print(psn)
            if psn != 'UTIL':
                ranked_position_df = self.projection.statline['batters'][(self.projection.statline['batters']['Position']==psn) & (self.projection.statline['batters']['Drafted']=='False')].sort_values('Rank')[0:self.number_teams]
                rank_stat = np.zeros([len(self.batter_statline),self.number_teams])

                for i in range(len(self.batter_statline)):
                    cat = self.batter_statline.keys()[i]
                    ind = np.argsort(ranked_position_df[cat])
                    if cat in ['HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']:
                        rank_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                position_rank = np.sum(rank_stat.T, axis=1)
                #print(position_rank)

                for i in (1 + np.arange(self.number_teams - 1)):
                    print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-i]], np.sort(position_rank)[-i])
                #print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1])
                
                df = pd.DataFrame([psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1]])
                if self.draftees.empty:
                    self.draftees = df.T
                else:
                    self.draftees = pd.concat([self.draftees, df.T])
                
        for psn in self.pitchers:
            if psn != 'P':
                ranked_position_df = self.projection.statline['pitchers'][(self.projection.statline['pitchers']['Position']==psn) & (self.projection.statline['pitchers']['Drafted']=='False')].sort_values('Rank')[0:self.number_teams]
                rank_stat = np.zeros([len(self.pitcher_statline),self.number_teams])

                for i in range(len(self.pitcher_statline)):
                    cat = self.pitcher_statline.keys()[i]
                    ind = np.argsort(ranked_position_df[cat])
                    if cat in ['SO', 'W', 'SV', 'ERA', 'CG', 'SHO']:
                        rank_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                    else:
                        rank_stat[i, ind]= np.arange(len(ind),0,-1)
                position_rank = np.sum(rank_stat.T, axis=1)
                for i in (1 + np.arange(self.number_teams - 1)):
                    print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-i]], np.sort(position_rank)[-i])

                #print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1])
                
                df = pd.DataFrame([psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1]])
                self.draftees = pd.concat([self.draftees, df.T])
                

In [50]:
ZiPS.statline['batters'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,playerid,Name,Drafted
0,76,79,32,4,25,84,5,38,0.276,0.838,507,C,2,11739,J.T. Realmuto,False
1,55,53,19,1,23,69,1,68,0.238,0.811,403,C,4,11368,Yasmani Grandal,False
2,68,53,21,0,28,85,3,44,0.246,0.822,415,C,1,11442,Gary Sanchez,False
3,53,89,24,1,8,53,4,48,0.288,0.770,423,C,5,9166,Buster Posey,False
4,56,75,26,3,14,64,5,51,0.255,0.754,463,C,6,11609,Willson Contreras,False


In [58]:
ZiPS_Dft = Draft(ZiPS,number_teams = 12)
Steam_Dft = Draft(Steamer,number_teams = 12)

['C', 'SS', 'OF', 'OF', 'OF', 'UTIL', '1B', '2B', '3B']
['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']
['Name', 'P', 'P', 'P', 'SP', 'SP', 'RP', 'RP']
['SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']
['C', 'SS', 'OF', 'OF', 'OF', 'UTIL', '1B', '2B', '3B']
['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']
['Name', 'P', 'P', 'P', 'SP', 'SP', 'RP', 'RP']
['SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']


In [59]:
Steam_Dft.draft_team()

('C', 'J.T. Realmuto', 100.0)
('C', 'Buster Posey', 92.0)
('C', 'Yadier Molina', 88.0)
('C', 'Gary Sanchez', 85.0)
('C', 'Willson Contreras', 77.0)
('C', 'Francisco Cervelli', 75.0)
('C', 'Yasmani Grandal', 68.0)
('C', 'Welington Castillo', 49.0)
('C', 'Danny Jansen', 47.0)
('C', 'Wilson Ramos', 39.0)
('C', 'Kurt Suzuki', 36.0)
('SS', 'Trevor Story', 84.0)
('SS', 'Francisco Lindor', 79.0)
('SS', 'Carlos Correa', 77.0)
('SS', 'Manny Machado', 77.0)
('SS', 'Xander Bogaerts', 73.0)
('SS', 'Trea Turner', 73.0)
('SS', 'Corey Seager', 69.0)
('SS', 'Javier Baez', 65.0)
('SS', 'Jean Segura', 58.0)
('SS', 'Jose Peraza', 56.0)
('SS', 'Adalberto Mondesi', 48.0)
('OF', 'Mookie Betts', 97.0)
('OF', 'Mike Trout', 96.0)
('OF', 'J.D. Martinez', 80.0)
('OF', 'Christian Yelich', 70.0)
('OF', 'Andrew Benintendi', 64.0)
('OF', 'Ronald Acuna Jr.', 62.0)
('OF', 'Charlie Blackmon', 62.0)
('OF', 'Aaron Judge', 58.0)
('OF', 'Bryce Harper', 55.0)
('OF', 'Starling Marte', 50.0)
('OF', 'Giancarlo Stanton', 44.0)


In [53]:
tstDft.draft_team()

('C', 'J.T. Realmuto', 108.0)
('C', 'Willson Contreras', 95.0)
('C', 'Buster Posey', 84.0)
('C', 'Gary Sanchez', 76.0)
('C', 'Yadier Molina', 74.0)
('C', 'Yasmani Grandal', 71.0)
('C', 'Danny Jansen', 64.0)
('C', 'Francisco Cervelli', 53.0)
('C', 'Wilson Ramos', 50.0)
('C', 'Yan Gomes', 43.0)
('C', 'Kurt Suzuki', 39.0)
('SS', 'Francisco Lindor', 95.0)
('SS', 'Manny Machado', 83.0)
('SS', 'Trea Turner', 80.0)
('SS', 'Xander Bogaerts', 78.0)
('SS', 'Trevor Story', 77.0)
('SS', 'Jean Segura', 67.0)
('SS', 'Javier Baez', 67.0)
('SS', 'Carlos Correa', 56.0)
('SS', 'Corey Seager', 55.0)
('SS', 'Jose Peraza', 52.0)
('SS', 'Adalberto Mondesi', 39.0)
('OF', 'Mookie Betts', 89.0)
('OF', 'Christian Yelich', 81.0)
('OF', 'Mike Trout', 81.0)
('OF', 'Bryce Harper', 74.0)
('OF', 'Charlie Blackmon', 74.0)
('OF', 'Andrew Benintendi', 67.0)
('OF', 'J.D. Martinez', 63.0)
('OF', 'Ronald Acuna Jr.', 58.0)
('OF', 'Rhys Hoskins', 54.0)
('OF', 'Giancarlo Stanton', 54.0)
('OF', 'Aaron Judge', 43.0)
('1B', 'Fre

In [253]:
batter_categories  = ['R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_categories = ['W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV' ]

psn = '3B'
ranked_position_df = bats_list[(bats_list['Position']==psn) & (bats_list['Drafted']==False)].sort_values('Rank')[0:12]
rank_stat = np.zeros([len(batter_categories),len(ind)])

for i in range(len(batter_categories)):
    cat = batter_categories[i]
    #print cat
    ind = np.argsort(ranked_position_df[cat])
    rank_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
print(psn,np.sum(rank_stat.T, axis=1))

('3B', array([ 91., 110.,  95.,  60.,  42.,  42.,  45.,  57.,  65.,  85.,  39.,
        49.]))


In [276]:
ranked_position_df.Name

1      Nolan Arenado
0       Jose Ramirez
3       Alex Bregman
7     Anthony Rendon
2     Josh Donaldson
12       Travis Shaw
14      Matt Chapman
6     Matt Carpenter
9     Eugenio Suarez
4        Kris Bryant
5      Justin Turner
15     Rafael Devers
Name: Name, dtype: object

In [11]:
number_positions = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}

In [12]:
print(number_positions.keys())
print(number_positions.values())
np.argsort(number_positions.values())[::-1] 

['C', 'BN', 'UTIL', 'P', '1B', '2B', '3B', 'RP', 'SS', 'OF', 'SP']
[1, 5, 1, 3, 1, 1, 1, 2, 1, 3, 2]


array([ 1,  9,  3, 10,  7,  8,  6,  5,  4,  2,  0])

In [13]:
ind = ZiPS.hitters_stats['Name'] == ZiPS.hitters_rank['C']['PLAYER'][0] 
ZiPS.hitters_stats[ind]
#ZiPS.pitchers_stats.head()

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,OPS,wOBA,Fld,BsR,WAR,ADP,playerid,Position,Rank,1B
2,Gary Sanchez,Yankees,110,467,415,102,21,0,28,68,...,0.822,0.35,-5.5,-0.1,3.1,55.3,11442,C,1,53


In [14]:
statline = {}
statline['batter_statline'] = {'R':0,'1B':0,'2B':0,'3B':0,'HR':0,'RBI':0,'SB':0,'BB':0,'AVG':0,'OPS':0}
statline['pitcher_statline'] = {'W':0,'L':0,'CG':0,'SHO':0,'SV':0,'BB':0,'K':0,'ERA':0,'WHIP':0,'BSV':0}

In [15]:
number_positions={'C':1,'1B':1,'2B':1,'3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}